In [1]:
!pip install dash pandas plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 55.3 MB/s eta 0:00:00


In [3]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# Sample dummy data
df = pd.DataFrame({
    "Date": pd.date_range(start="2023-01-01", periods=24, freq="ME"),  # Fixed freq
    "Region": ["North", "South", "East", "West"] * 6,
    "Product": ["A", "B", "C"] * 8,
    "Sales": [12000, 15000, 18000, 13000, 11000, 16000, 17500, 14000] * 3,
    "Profit": [3000, 4000, 5000, 3200, 2800, 4500, 4700, 3900] * 3
})
df["Month"] = df["Date"].dt.to_period("M").astype(str)

# App setup
app = dash.Dash(__name__)
app.title = "Business Dashboard"

# Layout
app.layout = html.Div(style={"font-family": "Arial", "padding": "20px"}, children=[
    html.H1("📊 Business Performance Dashboard", style={"textAlign": "center"}),

    # Filters
    html.Div([
        html.Div([
            html.Label("Select Region"),
            dcc.Dropdown(
                options=[{"label": r, "value": r} for r in df["Region"].unique()],
                value=df["Region"].unique()[0],
                id="region-filter"
            )
        ], style={"width": "30%", "display": "inline-block", "margin-right": "2%"}),

        html.Div([
            html.Label("Select Product"),
            dcc.Dropdown(
                options=[{"label": p, "value": p} for p in df["Product"].unique()],
                value=df["Product"].unique()[0],
                id="product-filter"
            )
        ], style={"width": "30%", "display": "inline-block"})
    ]),

    html.Br(),

    # KPI Cards
    html.Div(id="kpi-cards", style={"display": "flex", "justify-content": "space-between"}),

    html.Br(),

    # Time Series Chart
    dcc.Graph(id="sales-line-chart"),

    html.Br(),

    # Summary Table
    html.H3("📌 Monthly Summary"),
    dcc.Graph(id="monthly-summary")
])

# Callbacks
@app.callback(
    [Output("kpi-cards", "children"),
     Output("sales-line-chart", "figure"),
     Output("monthly-summary", "figure")],
    [Input("region-filter", "value"),
     Input("product-filter", "value")]
)
def update_dashboard(region, product):
    filtered_df = df[(df["Region"] == region) & (df["Product"] == product)]

    total_sales = filtered_df["Sales"].sum()
    total_profit = filtered_df["Profit"].sum()
    sales_growth = round(filtered_df["Sales"].pct_change().mean() * 100, 2)

    # KPI Cards
    cards = [
        html.Div(f"💰 Total Sales: ${total_sales:,.0f}", style=card_style()),
        html.Div(f"📈 Total Profit: ${total_profit:,.0f}", style=card_style()),
        html.Div(f"📊 Growth Rate: {sales_growth:.2f}%", style=card_style())
    ]

    # Line Chart
    line_fig = px.line(filtered_df, x="Date", y="Sales", title="Sales Trend Over Time")
    line_fig.update_layout(template="simple_white")

    # Summary Bar Chart
    summary = filtered_df.groupby("Month").agg({"Sales": "sum", "Profit": "sum"}).reset_index()
    summary_fig = px.bar(summary, x="Month", y=["Sales", "Profit"], barmode="group")
    summary_fig.update_layout(template="simple_white")

    return cards, line_fig, summary_fig

# Helper function for card style
def card_style():
    return {
        "padding": "20px",
        "border": "1px solid #ccc",
        "borderRadius": "10px",
        "width": "30%",
        "backgroundColor": "#f9f9f9",
        "textAlign": "center",
        "fontSize": "20px",
        "boxShadow": "2px 2px 10px rgba(0,0,0,0.1)"
    }

# Run app (Updated method)
if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>